# Notebook for OpenAI Pokemon Namen Generierung

Dieses Jupyter Notebook wurde entwickelt, um für jeden generierten synthetischen Datenpunkt einen passenden Pokémon-Namen zu generieren. Diese Generierung erfolgt mithilfe der OpenAI-Plattform und dient dazu, die Fähigkeiten von OpenAI im Bereich der kreativen Namensgebung zu demonstrieren.

#### Hintergrund

Die Generierung von Namen für synthetische Datenpunkte ist eine interessante Anwendung von KI-Technologien wie OpenAI. In diesem Notebook wird untersucht, wie gut OpenAI darin ist, Pokémon-Namen auf der Grundlage von gegebenen Statistiken zu erstellen. Dieser Ansatz ermöglicht es, automatisch und effizient eine große Anzahl von Namen zu generieren, die den vorgegebenen Kriterien entsprechen.

#### Ziel

Das Hauptziel dieses Notebooks besteht darin, die Qualität der von OpenAI generierten Pokémon-Namen zu bewerten. Hierbei wird auch ein Vergleich mit einem anderen LLM namens Llama 2 durchgeführt, um zu sehen, welches System bessere Namen erzeugt. 

## Imports

In [3]:
import os
import json
import pandas as pd

from llama_cpp import Llama
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI

In [4]:
# load api_keys from json
with open('api_key.json') as json_datei:
    keys = json.load(json_datei)

In [5]:
os.environ['OPENAI_API_KEY'] = keys["open_ai_api_key"] # set api key

## Code

### llama-index OpenAI

In [6]:
# Dieses Promptemaple definiert die Vorlage für die Generierung eines Pokémon-Namens auf der Grundlage der angegebenen Werte. (Sehr wichtig um Halluzination zu vermeiden!)
# Stats include: Name, Type 1, Type 2, Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed, Generation, Legendary, Evolution
# Examples:
# Input: Grass, Poison, 318, 45, 49, 49, 65, 65, 45, 1, False, 0; Output: Bulbasaur
# Input: Fire, NaN, 309, 39, 52, 43, 60, 50, 65, 1, False, 1; Output: Charmander
# Input: Fire, Water, 600, 80, 110, 120, 130, 90, 70, 6, True, 0; Output: Volcanion
# Only the name is output.
prompt_template= '<SYS> Generiere mir auf Basis dieser Stats einen Pokemon namen den es nicht gibt: Stats: Name, Type 1, Type 2, Total, HP, Attack, Defense, 	Sp. Atk,	Sp. Def,	Speed,	Generation,	Legendary, Evolution // Beispiele: Input:  Grass, Poison, 318, 45, 49,	49,	65,	65,	45,	1,	False, 0; Output: Bulbasaur Input: Fire, NaN,	309,	39,	52,	43,	60,	50,	65	1 False, 1; Output: Charmander Input: Fire, Water,	600,	80,	110,	120,	130,	90,	70,	6,	True, 0; Output: Volcanion	 Gib mir nur den Namen aus!'

In [7]:
def open_ai_pokemon_assistant(pokemon_stats):
    """ OpenAI Pokemon Assistant um Pokemon Namen anhand von einem String mit den Stats zu bewerten"""
    messages = [
        ChatMessage(role="system", content=prompt_template), # Systemnachricht mit dem vordefinierten prompt_template
        ChatMessage(role="user", content=pokemon_stats), # Pokemon_stats
    ]
    resp = OpenAI().chat(messages)
    return resp.message.content

In [8]:
#Datenvorbereinigung
syn_data = pd.read_csv("Gaussian_Pokemon.csv")
del syn_data["Unnamed: 0"]
syn_data = syn_data[0:10]
del syn_data["Name"]

# Wichtig um einen String zu generieren für das OpenAI Modul
syn_data['Attribut'] = syn_data['Type 1'] + ', ' + syn_data['Type 2'].astype(str) + ', ' + syn_data['HP'].astype(str) + ', ' + \
                        syn_data['Attack'].astype(str) + ', ' + syn_data['Defense'].astype(str) + ', ' + syn_data['Sp. Atk'].astype(str) + \
                        ', ' + syn_data['Sp. Def'].astype(str) + ', ' + syn_data['Speed'].astype(str) + ', ' + \
                        syn_data['Generation'].astype(str) + ', ' + syn_data['Legendary'].astype(str) + ', ' + \
                        syn_data['Evolution'].astype(str)

In [9]:
syn_data['Name_open_ai'] = syn_data["Attribut"].apply(open_ai_pokemon_assistant) # Assistant wird für jeden Datenpunkt (Pokemon) angewendet.
columns = ['Name_open_ai'] + [col for col in syn_data.columns if col != 'Name_open_ai'] # Name sollte ganz vorne stehen :)
syn_data = syn_data[columns]

In [10]:
syn_data.head(5)

,Name_open_ai,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Evolution,Attribut
0,Dragound,Ground,Dragon,81,93,99,88,83,71,6,False,0.0,"Ground, Dragon, 81, 93, 99, 88, 83, 71, 6, Fal..."
1,Aquarix,Water,NaN,32,24,29,43,34,39,1,False,1.0,"Water, nan, 32, 24, 29, 43, 34, 39, 1, False, 1.0"
2,Darklurk,Dark,NaN,45,28,22,35,27,71,5,False,1.0,"Dark, nan, 45, 28, 22, 35, 27, 71, 5, False, 1.0"
3,Flametrix,Fire,NaN,80,101,80,135,129,64,1,False,2.0,"Fire, nan, 80, 101, 80, 135, 129, 64, 1, False..."
4,Punchopponent,Fighting,NaN,57,74,43,71,71,79,3,False,1.0,"Fighting, nan, 57, 74, 43, 71, 71, 79, 3, Fals..."


## LLama

Das funktioniert leider überhaupt nicht gut, teilweise generiert er keinen Namen.

In [37]:
model_path = "llm/llama-2-7b-chat.Q2_K.gguf"
llm = Llama(model_path=model_path)
system_message = "Create a non-existent Pokemon name based on these statistics: Stats: Name, Type 1, Type 2, Total, HP, Attack, Defence, Sp. Atk, Sp. Def, Speed, Generation, Legendary, Evolution // Examples: Input: Grass, Poison, 318, 45, 49, 49, 65, 65, 45, 1, False, 0; Output: Bulbasaur Input: Fire, NaN, 309, 39, 52, 43, 60, 50, 65 1 False, 1; Output: Charmander Input: Fire, Water, 600, 80, 110, 120, 130, 90, 70, 6, True, 0; Output: Volcanic Ion; Give only the Name as Output!"

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llm/llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              

In [39]:
def llama_2_pokemon_assistant(pokemon_stats, llm=llm, system_message=system_message):
    user_message = f"Give me a pokemon name for these stats: {pokemon_stats}"
    prompt = f"""<s>[INST] <<SYS>> Create a non-existent Pokémon name based on the provided statistics. The statistics format includes: Name, Type 1, Type 2, Total, HP, Attack, Defence, Sp. Atk, Sp. Def, Speed, Generation, Legendary, Evolution. Examples of inputs and outputs are as follows:
- Input: Grass, Poison, 318, 45, 49, 49, 65, 65, 45, 1, False, 0; Output: Bulbasaur
- Input: Fire, NaN, 309, 39, 52, 43, 60, 50, 65, 1, False, 1; Output: Charmander
- Input: Fire, Water, 600, 80, 110, 120, 130, 90, 70, 6, True, 0; Output: Volcanic Ion [/SYS] [/INST]
[INST] Input: {user_message} Output: [/INST]</s>"""
    output = llm(
    prompt,
    max_tokens=40, # Generate up to 32 tokens
    #stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
    echo=True # Echo the prompt back in the output
    ) 
    return output["choices"][0]["text"].split("]")[-1]

In [40]:
syn_data['Name_llama_2'] = syn_data["Attribut"].apply(llama_2_pokemon_assistant) # Assistant wird für jeden Datenpunkt (Pokemon) angewendet.
columns = ['Name_llama_2'] + [col for col in syn_data.columns if col != 'Name_llama_2'] # Name sollte ganz vorne stehen :)
syn_data = syn_data[columns]

Llama.generate: prefix-match hit



llama_print_timings:        load time =   33740.20 ms
llama_print_timings:      sample time =      22.25 ms /    40 runs   (    0.56 ms per token,  1797.67 tokens per second)
llama_print_timings: prompt eval time =   43049.42 ms /   299 tokens (  143.98 ms per token,     6.95 tokens per second)
llama_print_timings:        eval time =   13801.11 ms /    39 runs   (  353.87 ms per token,     2.83 tokens per second)
llama_print_timings:       total time =   57036.86 ms /   338 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   33740.20 ms
llama_print_timings:      sample time =      20.96 ms /    40 runs   (    0.52 ms per token,  1907.94 tokens per second)
llama_print_timings: prompt eval time =   10782.22 ms /    44 tokens (  245.05 ms per token,     4.08 tokens per second)
llama_print_timings:        eval time =   13046.01 ms /    39 runs   (  334.51 ms per token,     2.99 tokens per second)
llama_print_timings:       total time =   24004.98 ms /    83 